In [1]:
import pathlib
import platform
from roboflow import Roboflow
plt = platform.system()
if plt != 'Windows': pathlib.WindowsPath = pathlib.PosixPath
rf = Roboflow(api_key="L92yodulFT4LlHfg1I5l")
project = rf.workspace("aiclimbing").project("difficulty-classifier")
dataset = project.version(2).download("folder")

loading Roboflow workspace...
loading Roboflow project...


TypeError: 'NoneType' object cannot be interpreted as an integer

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n-cls.pt')
results = model.train(data='../Difficulty Classifier Data', epochs=100, imgsz=64)

100%|█████████████████████████████████████████████████████████████████████████████| 5.28M/5.28M [00:00<00:00, 15.4MB/s]
New https://pypi.org/project/ultralytics/8.0.225 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196  Python-3.8.18 torch-2.1.1 CPU (Intel Core(TM) i7-10875H 2.30GHz)
engine\trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=../Difficulty Classifier Data, epochs=100, patience=50, batch=16, imgsz=64, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_c

               classes   top1_acc   top5_acc: 100%|██████████| 2/2 [00:00<00:00,  4.70it/s]
                   all      0.328       0.77

      Epoch    GPU_mem       loss  Instances       Size
     41/100         0G   0.002353          8         64: 100%|██████████| 89/89 [00:13<00:00,  6.47it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 2/2 [00:00<00:00,  4.82it/s]
                   all      0.311      0.787

      Epoch    GPU_mem       loss  Instances       Size
     42/100         0G   0.003124          8         64: 100%|██████████| 89/89 [00:13<00:00,  6.42it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 2/2 [00:00<00:00,  5.31it/s]
                   all      0.344      0.803

      Epoch    GPU_mem       loss  Instances       Size
     43/100         0G   0.003389          8         64: 100%|██████████| 89/89 [00:14<00:00,  6.21it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 2/2 [00:00<00:00,  4.91it/s]
           

In [2]:
import cv2
from ultralytics import YOLO
import pathlib
import platform
from roboflow import Roboflow
from PIL import Image
plt = platform.system()
# if plt != 'Windows': pathlib.WindowsPath = pathlib.PosixPath
# Load a pretrained YOLOv8n model
model = YOLO('../runs/classify/train4/weights/best.pt')
# Gets the image at the specified path as a numpy array
def getImage(path):
    img = cv2.imread(path)
    return img

# Displays the results of model(image)
def getResults(results):
    for r in results:
        guesses = r.probs.data
        if guesses[0] == max(guesses):
            return "bolt"
        if guesses[9] == max(guesses):
            return "tag"
        aggregate = (8 * guesses[2] + 5 * guesses[3] + 4 * guesses[4] + 9 * guesses[5] + 
                     1 * guesses[6] + 7 * guesses[7] + 6 * guesses[8] + 10 * guesses[10] + 3 * guesses[11] + 2 * guesses[12]) 
        return float(aggregate)
def getConfidence(results):
    for r in results:
        guesses = r.probs.data
        squares = 0
        for guess in guesses:
            squares += guess ** 2
    return float(squares)
def saveImage(image):
    im = Image.fromarray(image[..., ::-1])
    im.save('../images/results.jpg')
    
def rotateImage(mat, angle):
    height, width = mat.shape[:2] # image shape has 3 dimensions
    image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape
    
    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)
    
    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos = abs(rotation_mat[0,0]) 
    abs_sin = abs(rotation_mat[0,1])
    
    # find the new width and height bounds
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)
    
    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]
    
    # rotate image with the new bounds and translated rotation matrix
    rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h))
    return rotated_mat

def getIdealRotation(image):
    best = (0, 0)
    for i in range(8):
        rot = rotateImage(image, i * 45)
        res = model.predict(rot)
        conf = getConfidence(res)
        if conf > best[1]:
            best = (i * 45, conf)
    return best[0]

In [7]:
img = getImage("../images/blacktag.png")
print(getResults(model.predict(img)))
print(getIdealRotation(img))



0: 64x64 tag 0.84, eight 0.11, nine 0.01, seven 0.01, downclimb 0.01, 5.0ms
Speed: 1.8ms preprocess, 5.0ms inference, 0.1ms postprocess per image at shape (1, 3, 64, 64)
tag
